In [ ]:
import numpy as np
import pandas as pd
from pylab import mpl, plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set(style='whitegrid')
mpl.rcParams['font.family'] = 'serif'
%config InlineBackend.figure_format = 'svg'

In [ ]:
# Load financial data
raw = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data.csv', index_col=0, parse_dates=True)

In [ ]:
raw.info()

In [ ]:
raw.head()

In [ ]:
# Calculate log returns for all assets
returns = np.log(raw / raw.shift(1)).dropna()

In [ ]:
returns.head()

---

# PART 1: Portfolio Optimization with SciPy

**Modern Portfolio Theory (MPT)** seeks to construct portfolios that maximize return for a given level of risk, or minimize risk for a target return.

**Key Concepts:**
- **Expected Return**: Mean historical return of the portfolio
- **Portfolio Variance**: Weighted sum of asset variances and covariances
- **Sharpe Ratio**: Risk-adjusted return measure (excess return per unit of risk)
- **Efficient Frontier**: Set of optimal portfolios offering the best risk/return trade-off

We'll use **SciPy's optimization** to find the portfolio weights that maximize the Sharpe ratio.

## 1.1 Portfolio Statistics

First, let's define functions to calculate portfolio return, volatility, and Sharpe ratio.

In [ ]:
# Select assets for portfolio optimization
# Using a subset for clarity
portfolio_assets = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O']
print(f"Portfolio Assets: {portfolio_assets}")

In [ ]:
# Extract returns for selected assets
port_returns = returns[portfolio_assets].copy()

In [ ]:
# Calculate annualized statistics
mean_returns = port_returns.mean() * 252  # Annualized mean
cov_matrix = port_returns.cov() * 252     # Annualized covariance

print("Annualized Mean Returns:")
print(mean_returns)
print("\nAnnualized Covariance Matrix:")
print(cov_matrix)

In [ ]:
def portfolio_return(weights, mean_returns):
    """
    Calculate expected portfolio return.
    
    Parameters:
    -----------
    weights : array-like
        Asset weights (must sum to 1)
    mean_returns : array-like
        Expected return for each asset
    
    Returns:
    --------
    float : Portfolio expected return
    """
    return np.sum(weights * mean_returns)

In [ ]:
def portfolio_volatility(weights, cov_matrix):
    """
    Calculate portfolio volatility (standard deviation).
    
    Parameters:
    -----------
    weights : array-like
        Asset weights
    cov_matrix : DataFrame or array
        Covariance matrix of asset returns
    
    Returns:
    --------
    float : Portfolio volatility (annualized)
    """
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

In [ ]:
def portfolio_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.02):
    """
    Calculate portfolio Sharpe ratio.
    
    Sharpe Ratio = (Portfolio Return - Risk-Free Rate) / Portfolio Volatility
    
    Parameters:
    -----------
    weights : array-like
        Asset weights
    mean_returns : array-like
        Expected returns
    cov_matrix : DataFrame or array
        Covariance matrix
    risk_free_rate : float
        Risk-free rate (default 2% annually)
    
    Returns:
    --------
    float : Sharpe ratio
    """
    ret = portfolio_return(weights, mean_returns)
    vol = portfolio_volatility(weights, cov_matrix)
    return (ret - risk_free_rate) / vol

## 1.2 Test with Equal-Weight Portfolio

Before optimization, let's see how an equal-weight portfolio performs.

In [ ]:
# Equal weights for all assets
n_assets = len(portfolio_assets)
equal_weights = np.array([1/n_assets] * n_assets)

print(f"Equal Weights: {equal_weights}")
print(f"Sum of Weights: {equal_weights.sum()}")

In [ ]:
# Calculate equal-weight portfolio statistics
eq_return = portfolio_return(equal_weights, mean_returns)
eq_volatility = portfolio_volatility(equal_weights, cov_matrix)
eq_sharpe = portfolio_sharpe_ratio(equal_weights, mean_returns, cov_matrix)

print(f"Equal-Weight Portfolio:")
print(f"{'='*60}")
print(f"Expected Return:  {eq_return:.2%}")
print(f"Volatility:       {eq_volatility:.2%}")
print(f"Sharpe Ratio:     {eq_sharpe:.3f}")

## 1.3 Optimize for Maximum Sharpe Ratio

Now we'll use **SciPy's minimize** function to find the optimal weights.

**Optimization Problem:**
- **Objective**: Maximize Sharpe ratio (minimize negative Sharpe ratio)
- **Constraints**: Weights sum to 1
- **Bounds**: Each weight between 0 and 1 (long-only portfolio)

In [ ]:
from scipy.optimize import minimize

In [ ]:
def negative_sharpe(weights, mean_returns, cov_matrix, risk_free_rate=0.02):
    """
    Negative Sharpe ratio for minimization.
    
    SciPy's minimize function finds minimum values, so we negate the Sharpe ratio.
    """
    return -portfolio_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate)

In [ ]:
# Define constraints and bounds
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Weights sum to 1
bounds = tuple((0, 1) for _ in range(n_assets))  # Each weight between 0 and 1

# Initial guess (equal weights)
init_guess = equal_weights

print("Optimization Setup:")
print(f"Number of assets: {n_assets}")
print(f"Constraint: Weights sum to 1")
print(f"Bounds: Each weight in [0, 1] (long-only)")

In [ ]:
# Run optimization
result = minimize(
    negative_sharpe,
    init_guess,
    args=(mean_returns, cov_matrix),
    method='SLSQP',  # Sequential Least Squares Programming
    bounds=bounds,
    constraints=constraints
)

# Extract optimal weights
optimal_weights = result.x

print("\nOptimization Result:")
print(f"Success: {result.success}")
print(f"Message: {result.message}")

In [ ]:
# Display optimal portfolio
optimal_portfolio = pd.DataFrame({
    'Asset': portfolio_assets,
    'Optimal Weight': optimal_weights,
    'Equal Weight': equal_weights
})
optimal_portfolio = optimal_portfolio.sort_values('Optimal Weight', ascending=False)
optimal_portfolio

In [ ]:
# Calculate optimal portfolio statistics
opt_return = portfolio_return(optimal_weights, mean_returns)
opt_volatility = portfolio_volatility(optimal_weights, cov_matrix)
opt_sharpe = portfolio_sharpe_ratio(optimal_weights, mean_returns, cov_matrix)

print(f"Optimal Portfolio (Maximum Sharpe Ratio):")
print(f"{'='*60}")
print(f"Expected Return:  {opt_return:.2%}")
print(f"Volatility:       {opt_volatility:.2%}")
print(f"Sharpe Ratio:     {opt_sharpe:.3f}")

## 1.4 Compare Portfolios

In [ ]:
# Create comparison table
comparison = pd.DataFrame({
    'Metric': ['Expected Return', 'Volatility', 'Sharpe Ratio'],
    'Equal-Weight': [eq_return, eq_volatility, eq_sharpe],
    'Optimized': [opt_return, opt_volatility, opt_sharpe]
})

comparison['Improvement'] = comparison['Optimized'] - comparison['Equal-Weight']
comparison

In [ ]:
# Visualize weight allocation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Equal weights
axes[0].bar(portfolio_assets, equal_weights, color='steelblue', edgecolor='black')
axes[0].set_title('Equal-Weight Portfolio', fontsize=13, pad=15)
axes[0].set_ylabel('Weight', fontsize=11)
axes[0].set_ylim(0, 1)
axes[0].grid(True, alpha=0.3)

# Optimal weights
axes[1].bar(portfolio_assets, optimal_weights, color='darkgreen', edgecolor='black')
axes[1].set_title('Optimal Portfolio (Max Sharpe)', fontsize=13, pad=15)
axes[1].set_ylabel('Weight', fontsize=11)
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 1.5 Efficient Frontier Visualization

The **efficient frontier** shows all optimal portfolios across different risk levels.

In [ ]:
# Generate random portfolios
n_portfolios = 10000
results = np.zeros((3, n_portfolios))  # Store return, volatility, Sharpe

np.random.seed(42)

for i in range(n_portfolios):
    # Random weights that sum to 1
    weights = np.random.random(n_assets)
    weights /= weights.sum()
    
    # Calculate portfolio stats
    results[0, i] = portfolio_return(weights, mean_returns)
    results[1, i] = portfolio_volatility(weights, cov_matrix)
    results[2, i] = portfolio_sharpe_ratio(weights, mean_returns, cov_matrix)

print(f"Generated {n_portfolios} random portfolios")

In [ ]:
# Plot efficient frontier
plt.figure(figsize=(12, 7))

# Scatter plot of random portfolios (color by Sharpe ratio)
scatter = plt.scatter(
    results[1, :], 
    results[0, :],
    c=results[2, :],
    cmap='viridis',
    marker='o',
    s=10,
    alpha=0.5
)

# Highlight optimal portfolio
plt.scatter(
    opt_volatility,
    opt_return,
    c='red',
    marker='*',
    s=500,
    edgecolors='black',
    label='Optimal Portfolio',
    zorder=5
)

# Highlight equal-weight portfolio
plt.scatter(
    eq_volatility,
    eq_return,
    c='orange',
    marker='D',
    s=200,
    edgecolors='black',
    label='Equal-Weight Portfolio',
    zorder=5
)

plt.colorbar(scatter, label='Sharpe Ratio')
plt.xlabel('Volatility (Risk)', fontsize=12)
plt.ylabel('Expected Return', fontsize=12)
plt.title('Efficient Frontier - Risk vs Return', fontsize=14, pad=20)
plt.legend(fontsize=10, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Key Insights from Portfolio Optimization

**What we learned:**
1. **Diversification isn't always equal-weight** - Optimal allocation depends on:
   - Individual asset expected returns
   - Asset volatilities
   - Correlation structure between assets

2. **The efficient frontier** shows the trade-off between risk and return
   - Portfolios below the frontier are sub-optimal
   - The optimal portfolio maximizes Sharpe ratio (best risk-adjusted return)

3. **Practical considerations:**
   - Historical data may not predict future returns
   - Transaction costs and constraints matter
   - Rebalancing frequency affects performance
   - Consider adding constraints (sector limits, turnover constraints)

**Analysis Questions:**
1. Which asset received the highest weight? Why might that be?
2. Are any assets completely excluded (weight ≈ 0)? What does this suggest?
3. How sensitive is the optimal portfolio to the sample period used?

---

# PART 2: Cointegration Testing for Pairs Trading

**Cointegration** is a statistical property indicating that two or more time series move together in the long run, even if they drift apart temporarily.

**Why it matters for trading:**
- Identifies pairs trading opportunities
- Mean-reverting spreads can be traded
- Statistical arbitrage strategies rely on cointegration
- Provides hedge ratios for portfolio construction

**Augmented Dickey-Fuller (ADF) Test:**
- Tests whether a time series is stationary
- For pairs trading: test if the spread between two assets is stationary
- **Null hypothesis**: Series has a unit root (non-stationary)
- **Alternative**: Series is stationary (mean-reverting)

## 2.1 Understanding Stationarity

A stationary time series has:
- Constant mean over time
- Constant variance over time
- No trend or seasonality

**For pairs trading:** We want the spread to be stationary (mean-reverting).

In [ ]:
# Select two assets for cointegration analysis
asset1 = 'AAPL.O'
asset2 = 'MSFT.O'

print(f"Testing cointegration between {asset1} and {asset2}")

In [ ]:
# Extract price series
prices_pair = raw[[asset1, asset2]].dropna()

In [ ]:
# Visualize price series
prices_pair.plot(figsize=(12, 6), linewidth=2)
plt.title(f'Price Series: {asset1} vs {asset2}', fontsize=14, pad=20)
plt.ylabel('Price', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Question: Do these prices appear to move together?")

## 2.2 Perform Linear Regression

First, we need to find the hedge ratio (beta) between the two assets using linear regression.

The hedge ratio tells us how many units of asset2 to short for each unit of asset1 we long.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Fit linear regression: asset1 = alpha + beta * asset2
X = prices_pair[asset2].values.reshape(-1, 1)
y = prices_pair[asset1].values

model = LinearRegression()
model.fit(X, y)

hedge_ratio = model.coef_[0]
intercept = model.intercept_

print(f"Linear Regression Results:")
print(f"{'='*60}")
print(f"Hedge Ratio (Beta): {hedge_ratio:.4f}")
print(f"Intercept (Alpha):  {intercept:.4f}")
print(f"\nInterpretation: For every $1 change in {asset2},")
print(f"{asset1} changes by ${hedge_ratio:.4f}")

In [ ]:
# Visualize regression
plt.figure(figsize=(10, 6))
plt.scatter(prices_pair[asset2], prices_pair[asset1], alpha=0.3, s=10)
plt.plot(
    prices_pair[asset2],
    intercept + hedge_ratio * prices_pair[asset2],
    'r-',
    linewidth=2,
    label=f'Regression Line (β={hedge_ratio:.3f})'
)
plt.xlabel(f'{asset2} Price', fontsize=12)
plt.ylabel(f'{asset1} Price', fontsize=12)
plt.title('Price Relationship & Hedge Ratio', fontsize=14, pad=20)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2.3 Calculate Spread

The **spread** is the residual from the regression:

**Spread = asset1 - (hedge_ratio × asset2)**

If the assets are cointegrated, this spread should be stationary (mean-reverting).

In [ ]:
# Calculate spread
prices_pair['spread'] = prices_pair[asset1] - hedge_ratio * prices_pair[asset2]

In [ ]:
# Visualize spread
prices_pair['spread'].plot(figsize=(12, 6), linewidth=1.5, color='purple')
plt.axhline(prices_pair['spread'].mean(), color='red', linestyle='--', label='Mean')
plt.axhline(
    prices_pair['spread'].mean() + prices_pair['spread'].std(),
    color='orange',
    linestyle='--',
    alpha=0.7,
    label='+1 Std Dev'
)
plt.axhline(
    prices_pair['spread'].mean() - prices_pair['spread'].std(),
    color='orange',
    linestyle='--',
    alpha=0.7,
    label='-1 Std Dev'
)
plt.title(f'Spread: {asset1} - {hedge_ratio:.3f} × {asset2}', fontsize=14, pad=20)
plt.ylabel('Spread Value', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Visual test: Does the spread oscillate around the mean?")
print("This suggests potential mean-reversion (stationarity).")

## 2.4 Augmented Dickey-Fuller (ADF) Test

Now we formally test if the spread is stationary using the ADF test.

**Interpretation:**
- **p-value < 0.05**: Reject null hypothesis → Spread is stationary → Cointegration exists! ✓
- **p-value ≥ 0.05**: Cannot reject null → Spread is non-stationary → No cointegration ✗

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def adf_test(series, name=''):
    """
    Perform Augmented Dickey-Fuller test on a time series.
    
    Parameters:
    -----------
    series : array-like
        Time series to test
    name : str
        Name of the series for display
    
    Returns:
    --------
    dict : Test results including test statistic, p-value, and critical values
    """
    result = adfuller(series, autolag='AIC')
    
    print(f'ADF Test Results for {name}')
    print(f"{'='*60}")
    print(f'ADF Test Statistic:    {result[0]:.4f}')
    print(f'p-value:               {result[1]:.4f}')
    print(f'Number of Lags Used:   {result[2]}')
    print(f'Number of Observations: {result[3]}')
    print(f'\nCritical Values:')
    for key, value in result[4].items():
        print(f'  {key}: {value:.4f}')
    
    # Interpretation
    print(f'\n{"="*60}')
    if result[1] < 0.05:
        print(f'✓ RESULT: Reject null hypothesis (p-value = {result[1]:.4f})')
        print(f'✓ The series is STATIONARY (mean-reverting)')
        print(f'✓ Cointegration likely exists!')
    else:
        print(f'✗ RESULT: Cannot reject null hypothesis (p-value = {result[1]:.4f})')
        print(f'✗ The series is NON-STATIONARY')
        print(f'✗ No evidence of cointegration')
    
    return {
        'test_statistic': result[0],
        'p_value': result[1],
        'lags_used': result[2],
        'n_obs': result[3],
        'critical_values': result[4],
        'is_stationary': result[1] < 0.05
    }

In [ ]:
# Test the spread for stationarity
adf_result = adf_test(prices_pair['spread'].dropna(), name='Spread')

## 2.5 Test Individual Assets (Comparison)

For comparison, let's test if the individual asset prices are stationary.

**Expected:** Individual stock prices are typically non-stationary (they have trends).

In [ ]:
# Test asset1
print("\n" + "="*60)
adf_result1 = adf_test(prices_pair[asset1], name=asset1)

In [ ]:
# Test asset2
print("\n" + "="*60)
adf_result2 = adf_test(prices_pair[asset2], name=asset2)

### Key Insight

**Cointegration vs Correlation:**
- **Correlation** measures linear relationship at a point in time
- **Cointegration** measures long-run equilibrium relationship
- Two assets can be cointegrated but have low short-term correlation
- Cointegration is more useful for pairs trading than correlation

## 2.6 Test Multiple Pairs

Let's test cointegration across multiple asset pairs to find the best candidates.

In [ ]:
# Select assets for pairwise testing
test_assets = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GOOGL.O']

In [ ]:
# Create function to test all pairs
def test_all_pairs(assets, prices_data):
    """
    Test cointegration for all possible pairs.
    
    Returns DataFrame with test results for each pair.
    """
    results = []
    
    for i in range(len(assets)):
        for j in range(i+1, len(assets)):
            asset_i = assets[i]
            asset_j = assets[j]
            
            # Get prices
            pair_data = prices_data[[asset_i, asset_j]].dropna()
            
            # Run regression
            X = pair_data[asset_j].values.reshape(-1, 1)
            y = pair_data[asset_i].values
            
            reg = LinearRegression()
            reg.fit(X, y)
            
            hedge = reg.coef_[0]
            
            # Calculate spread
            spread = pair_data[asset_i] - hedge * pair_data[asset_j]
            
            # ADF test
            adf_result = adfuller(spread, autolag='AIC')
            
            results.append({
                'Asset 1': asset_i,
                'Asset 2': asset_j,
                'Hedge Ratio': hedge,
                'ADF Statistic': adf_result[0],
                'p-value': adf_result[1],
                'Is Cointegrated': adf_result[1] < 0.05
            })
    
    return pd.DataFrame(results)

In [ ]:
# Test all pairs
cointegration_results = test_all_pairs(test_assets, raw)
cointegration_results = cointegration_results.sort_values('p-value')
cointegration_results

In [ ]:
# Summarize findings
n_pairs = len(cointegration_results)
n_cointegrated = cointegration_results['Is Cointegrated'].sum()

print(f"Cointegration Testing Summary:")
print(f"{'='*60}")
print(f"Total pairs tested:      {n_pairs}")
print(f"Cointegrated pairs:      {n_cointegrated}")
print(f"Percentage cointegrated: {n_cointegrated/n_pairs:.1%}")
print(f"\nBest pair (lowest p-value):")
best_pair = cointegration_results.iloc[0]
print(f"  {best_pair['Asset 1']} & {best_pair['Asset 2']}")
print(f"  p-value: {best_pair['p-value']:.4f}")
print(f"  Hedge ratio: {best_pair['Hedge Ratio']:.4f}")

---

# PART 3: Portfolio Beta & Z-Scores

**Portfolio Beta** measures a portfolio's sensitivity to market movements.

**Z-Score** measures how many standard deviations a value is from the mean - crucial for pairs trading signals.

These metrics are essential for:
- Risk management (market exposure)
- Position sizing
- Trading signal generation
- Portfolio hedging

## 3.1 Calculate Portfolio Beta

**Beta (β)** measures systematic risk:
- **β = 1**: Portfolio moves with the market
- **β > 1**: Portfolio is more volatile than market (aggressive)
- **β < 1**: Portfolio is less volatile than market (defensive)
- **β < 0**: Portfolio moves opposite to market (hedged)

**Formula:** β = Cov(Portfolio, Market) / Var(Market)

In [ ]:
# Use SPY (S&P 500 ETF) as market proxy
# We'll use .SPX (S&P 500 Index) from our data
market_symbol = '.SPX'

# Check if market index is available
if market_symbol in raw.columns:
    market_prices = raw[market_symbol]
    market_returns = np.log(market_prices / market_prices.shift(1)).dropna()
    print(f"Using {market_symbol} as market benchmark")
else:
    # Fallback: use an equity instrument
    market_symbol = 'AAPL.O'
    market_prices = raw[market_symbol]
    market_returns = np.log(market_prices / market_prices.shift(1)).dropna()
    print(f"Using {market_symbol} as market proxy (SPX not available)")

In [ ]:
# Create portfolio returns using optimal weights
portfolio_returns = (port_returns * optimal_weights).sum(axis=1)

In [ ]:
# Align portfolio and market returns
aligned_data = pd.DataFrame({
    'portfolio': portfolio_returns,
    'market': market_returns
}).dropna()

aligned_data.head()

In [ ]:
def calculate_beta(portfolio_returns, market_returns):
    """
    Calculate portfolio beta.
    
    Beta = Covariance(Portfolio, Market) / Variance(Market)
    
    Parameters:
    -----------
    portfolio_returns : array-like
        Portfolio return series
    market_returns : array-like
        Market return series
    
    Returns:
    --------
    dict : Beta, correlation, and related statistics
    """
    # Calculate covariance and variance
    cov_matrix = np.cov(portfolio_returns, market_returns)
    covariance = cov_matrix[0, 1]
    market_variance = cov_matrix[1, 1]
    
    # Calculate beta
    beta = covariance / market_variance
    
    # Calculate correlation
    correlation = np.corrcoef(portfolio_returns, market_returns)[0, 1]
    
    # Calculate alpha (excess return not explained by market)
    # Using simple regression
    from sklearn.linear_model import LinearRegression
    X = market_returns.values.reshape(-1, 1)
    y = portfolio_returns.values
    
    reg = LinearRegression()
    reg.fit(X, y)
    
    alpha = reg.intercept_ * 252  # Annualized
    
    return {
        'beta': beta,
        'correlation': correlation,
        'alpha': alpha,
        'covariance': covariance,
        'market_variance': market_variance
    }

In [ ]:
# Calculate beta for optimal portfolio
beta_stats = calculate_beta(aligned_data['portfolio'], aligned_data['market'])

print(f"Portfolio Beta Analysis:")
print(f"{'='*60}")
print(f"Beta:                {beta_stats['beta']:.4f}")
print(f"Correlation:         {beta_stats['correlation']:.4f}")
print(f"Alpha (annualized):  {beta_stats['alpha']:.2%}")
print(f"\nInterpretation:")
if beta_stats['beta'] > 1:
    print(f"  Portfolio is MORE volatile than the market")
    print(f"  A 1% market move → {beta_stats['beta']:.2%} portfolio move (expected)")
elif beta_stats['beta'] < 1 and beta_stats['beta'] > 0:
    print(f"  Portfolio is LESS volatile than the market")
    print(f"  A 1% market move → {beta_stats['beta']:.2%} portfolio move (expected)")
else:
    print(f"  Portfolio has negative or zero market correlation")

In [ ]:
# Visualize portfolio vs market relationship
plt.figure(figsize=(10, 6))
plt.scatter(aligned_data['market'], aligned_data['portfolio'], alpha=0.3, s=10)

# Add regression line
from sklearn.linear_model import LinearRegression
X = aligned_data['market'].values.reshape(-1, 1)
y = aligned_data['portfolio'].values
reg = LinearRegression()
reg.fit(X, y)

plt.plot(
    aligned_data['market'],
    reg.predict(X),
    'r-',
    linewidth=2,
    label=f'β = {beta_stats["beta"]:.3f}'
)

plt.xlabel('Market Returns', fontsize=12)
plt.ylabel('Portfolio Returns', fontsize=12)
plt.title('Portfolio Beta - Relationship with Market', fontsize=14, pad=20)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.2 Calculate Z-Scores for Pairs Trading

**Z-Score** measures how far the spread has deviated from its mean in terms of standard deviations.

**Formula:** Z = (Spread - Mean) / Std Dev

**Trading signals:**
- **Z > +2**: Spread is too high → SHORT the spread (short asset1, long asset2)
- **Z < -2**: Spread is too low → LONG the spread (long asset1, short asset2)
- **|Z| < 0.5**: Spread near mean → CLOSE positions

In [ ]:
# Use the best cointegrated pair from earlier
best_pair = cointegration_results.iloc[0]
asset1_pair = best_pair['Asset 1']
asset2_pair = best_pair['Asset 2']
hedge_ratio_pair = best_pair['Hedge Ratio']

print(f"Analyzing pair: {asset1_pair} & {asset2_pair}")
print(f"Hedge Ratio: {hedge_ratio_pair:.4f}")

In [ ]:
# Get prices and calculate spread
pair_prices = raw[[asset1_pair, asset2_pair]].dropna()
pair_prices['spread'] = pair_prices[asset1_pair] - hedge_ratio_pair * pair_prices[asset2_pair]

In [ ]:
def calculate_zscore(series, window=20):
    """
    Calculate rolling z-score.
    
    Z-score = (Value - Rolling Mean) / Rolling Std Dev
    
    Parameters:
    -----------
    series : array-like
        Time series data
    window : int
        Rolling window size
    
    Returns:
    --------
    Series : Z-scores
    """
    rolling_mean = series.rolling(window=window).mean()
    rolling_std = series.rolling(window=window).std()
    zscore = (series - rolling_mean) / rolling_std
    return zscore

In [ ]:
# Calculate z-scores with different windows
pair_prices['zscore_20'] = calculate_zscore(pair_prices['spread'], window=20)
pair_prices['zscore_60'] = calculate_zscore(pair_prices['spread'], window=60)

In [ ]:
pair_prices[['spread', 'zscore_20', 'zscore_60']].tail(10)

In [ ]:
# Visualize z-scores with trading signals
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot 1: Spread
axes[0].plot(pair_prices.index, pair_prices['spread'], linewidth=1, color='purple')
axes[0].axhline(pair_prices['spread'].mean(), color='black', linestyle='--', label='Mean')
axes[0].set_ylabel('Spread Value', fontsize=11)
axes[0].set_title(f'Spread: {asset1_pair} - {hedge_ratio_pair:.3f} × {asset2_pair}', fontsize=13)
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3)

# Plot 2: Z-Scores with trading signals
axes[1].plot(pair_prices.index, pair_prices['zscore_20'], linewidth=1, label='Z-Score (20-day)', color='blue')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.8)
axes[1].axhline(2, color='red', linestyle='--', linewidth=1.5, label='Overbought (+2σ)')
axes[1].axhline(-2, color='green', linestyle='--', linewidth=1.5, label='Oversold (-2σ)')
axes[1].axhline(1, color='orange', linestyle=':', alpha=0.7)
axes[1].axhline(-1, color='orange', linestyle=':', alpha=0.7)
axes[1].fill_between(pair_prices.index, 2, 3, alpha=0.2, color='red')
axes[1].fill_between(pair_prices.index, -2, -3, alpha=0.2, color='green')
axes[1].set_ylabel('Z-Score', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_title('Z-Score with Trading Signals', fontsize=13)
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-3, 3)

plt.tight_layout()
plt.show()

## 3.3 Generate Trading Signals

Create actual entry/exit signals based on z-scores.

In [ ]:
# Define signal thresholds
entry_threshold = 2.0  # Enter when |z| > 2
exit_threshold = 0.5   # Exit when |z| < 0.5

# Generate signals
pair_prices['signal'] = 0

# Long spread signal (z-score < -2): Long asset1, Short asset2
pair_prices.loc[pair_prices['zscore_20'] < -entry_threshold, 'signal'] = 1

# Short spread signal (z-score > 2): Short asset1, Long asset2
pair_prices.loc[pair_prices['zscore_20'] > entry_threshold, 'signal'] = -1

# Maintain position until exit threshold
pair_prices['position'] = pair_prices['signal'].replace(0, np.nan).fillna(method='ffill').fillna(0)

# Exit when z-score crosses exit threshold
exit_condition = np.abs(pair_prices['zscore_20']) < exit_threshold
pair_prices.loc[exit_condition, 'position'] = 0

In [ ]:
# Count signals
signal_counts = pair_prices['signal'].value_counts()
print(f"Trading Signals Generated:")
print(f"{'='*60}")
print(f"Long spread signals:   {signal_counts.get(1, 0)}")
print(f"Short spread signals:  {signal_counts.get(-1, 0)}")
print(f"Neutral periods:       {signal_counts.get(0, 0)}")

In [ ]:
# Calculate strategy returns
pair_returns = pair_prices[[asset1_pair, asset2_pair]].pct_change()

# Spread return = return on asset1 - hedge_ratio * return on asset2
pair_prices['spread_return'] = (
    pair_returns[asset1_pair] - hedge_ratio_pair * pair_returns[asset2_pair]
)

# Strategy return = position * spread_return
pair_prices['strategy_return'] = pair_prices['position'].shift(1) * pair_prices['spread_return']

In [ ]:
# Performance metrics
cumulative_strategy = (1 + pair_prices['strategy_return'].fillna(0)).cumprod()
total_return = cumulative_strategy.iloc[-1] - 1

# Sharpe ratio
sharpe = (pair_prices['strategy_return'].mean() / pair_prices['strategy_return'].std()) * np.sqrt(252)

# Max drawdown
cummax = cumulative_strategy.cummax()
drawdown = (cumulative_strategy - cummax) / cummax
max_drawdown = drawdown.min()

print(f"Pairs Trading Strategy Performance:")
print(f"{'='*60}")
print(f"Total Return:     {total_return:.2%}")
print(f"Sharpe Ratio:     {sharpe:.3f}")
print(f"Max Drawdown:     {max_drawdown:.2%}")

In [ ]:
# Plot cumulative returns
plt.figure(figsize=(12, 6))
cumulative_strategy.plot(linewidth=2, color='darkgreen')
plt.title('Pairs Trading Strategy - Cumulative Returns', fontsize=14, pad=20)
plt.ylabel('Cumulative Return', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.4 Z-Score Statistics

Analyze the distribution and properties of z-scores.

In [ ]:
# Z-score statistics
zscore_stats = pair_prices['zscore_20'].dropna().describe()
print("Z-Score Statistics:")
print(zscore_stats)

In [ ]:
# Plot z-score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(pair_prices['zscore_20'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='black', linestyle='--', linewidth=1.5, label='Mean (0)')
axes[0].axvline(2, color='red', linestyle='--', linewidth=1.5, label='+2σ')
axes[0].axvline(-2, color='green', linestyle='--', linewidth=1.5, label='-2σ')
axes[0].set_xlabel('Z-Score', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Z-Score Distribution', fontsize=13)
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3)

# Time series of absolute z-scores
axes[1].plot(pair_prices.index, np.abs(pair_prices['zscore_20']), linewidth=1, color='purple', alpha=0.6)
axes[1].axhline(2, color='red', linestyle='--', linewidth=1.5, label='Entry Threshold (2σ)')
axes[1].axhline(0.5, color='orange', linestyle='--', linewidth=1.5, label='Exit Threshold (0.5σ)')
axes[1].set_ylabel('|Z-Score|', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_title('Absolute Z-Score Over Time', fontsize=13)
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate percentage of time at different z-score levels
zscore_abs = np.abs(pair_prices['zscore_20'].dropna())

pct_extreme = (zscore_abs > 2).sum() / len(zscore_abs) * 100
pct_moderate = ((zscore_abs > 1) & (zscore_abs <= 2)).sum() / len(zscore_abs) * 100
pct_normal = (zscore_abs <= 1).sum() / len(zscore_abs) * 100

print(f"Z-Score Time Distribution:")
print(f"{'='*60}")
print(f"|Z| > 2 (Extreme):     {pct_extreme:.1f}%")
print(f"1 < |Z| ≤ 2 (Moderate): {pct_moderate:.1f}%")
print(f"|Z| ≤ 1 (Normal):      {pct_normal:.1f}%")
print(f"\nTrading opportunities (|Z| > 2) occur {pct_extreme:.1f}% of the time")

---

## Summary: Key Takeaways

### Portfolio Optimization:
1. **Mean-Variance Optimization** - Mathematical framework for optimal allocation
   - Maximizes Sharpe ratio (risk-adjusted return)
   - Considers correlations between assets
   - SciPy provides powerful optimization tools
   
2. **Efficient Frontier** - Visualizes risk-return trade-offs
   - Only portfolios on the frontier are optimal
   - Different risk preferences → different optimal portfolios
   - Real-world constraints matter (transaction costs, limits)

3. **Equal-weight vs Optimized** - Optimization can improve performance
   - But: based on historical data (may not predict future)
   - Requires periodic rebalancing
   - Consider estimation error and overfitting

### Cointegration Testing:
1. **ADF Test** - Statistical test for mean-reversion
   - Tests if spread is stationary
   - p-value < 0.05 indicates cointegration
   - More robust than correlation for pairs trading
   
2. **Hedge Ratio** - Determines proper position sizes
   - From linear regression
   - Creates a mean-reverting spread
   - Should be monitored and updated

3. **Practical Considerations**
   - Cointegration can break down over time
   - Transaction costs matter for high-frequency trading
   - Need sufficient capital to hold both legs

### Portfolio Beta & Z-Scores:
1. **Beta** - Measures market exposure
   - β > 1: Aggressive (amplifies market moves)
   - β < 1: Defensive (dampens market moves)
   - Critical for risk management and hedging
   
2. **Z-Scores** - Trading signal generation
   - Standardizes deviations from mean
   - Entry signals at ±2 standard deviations
   - Exit signals when returning to mean
   
3. **Applications**
   - Beta: Portfolio hedging, risk budgeting
   - Z-scores: Pairs trading, mean reversion strategies
   - Both: Essential risk management tools

### Best Practices:
✅ **DO:**
- Update optimization inputs regularly
- Test multiple cointegration periods
- Monitor beta drift over time
- Use multiple confirmation signals
- Account for transaction costs

❌ **DON'T:**
- Assume historical patterns persist forever
- Over-optimize on in-sample data
- Ignore regime changes
- Trade pairs without stop-losses
- Neglect position sizing

### Integration:
These three techniques work together:
- **Optimization** → Find best portfolio weights
- **Cointegration** → Identify pairs for hedging or arbitrage
- **Beta & Z-scores** → Manage risk and generate signals

---

## 📝 Student Assignment

Complete the following tasks to apply portfolio optimization, cointegration testing, and analytics.

**Expected Time**: 90-120 minutes

---

## Task 1: Constrained Portfolio Optimization

**Objective:** Add realistic constraints to portfolio optimization and compare results.

Implement at least TWO of the following constraint scenarios:

### Option A: Sector/Asset Limits

Add maximum weight constraints (e.g., no asset > 40% of portfolio).

In [ ]:
# TODO: Implement optimization with individual asset constraints
# Hint: Modify bounds parameter or add additional constraints
# Example: No asset weight > 0.40 (40%)

# Your code here

### Option B: Long-Short Portfolio

Allow short positions (negative weights) within limits.

In [ ]:
# TODO: Implement long-short optimization
# Hint: Change bounds to allow negative weights (e.g., -0.5 to 1.0)
# Add constraint that absolute weights sum to 1.0

# Your code here

### Option C: Minimum Weight Requirements

Require minimum allocation to each asset (e.g., at least 10% per asset).

In [ ]:
# TODO: Implement optimization with minimum weights
# Hint: Set bounds like (0.10, 0.50) instead of (0, 1)

# Your code here

In [ ]:
# TODO: Compare all portfolio variants
# Create comparison table showing:
# - Weights for each portfolio
# - Expected return
# - Volatility  
# - Sharpe ratio
# - Sum of absolute weights (for long-short)

# Your code here

### Analysis Questions for Task 1

**TODO: Write your analysis here**

1. How do constraints affect the optimal portfolio?
2. Which constrained portfolio would you prefer for real trading? Why?
3. What's the trade-off between constraints and performance?
4. For long-short portfolios: How does allowing shorts change the efficient frontier?

---

## Task 2: Rolling Cointegration Analysis

**Objective:** Analyze how cointegration relationships change over time.

Test if cointegrated pairs remain stable.

In [ ]:
# TODO: Implement rolling cointegration test
# 1. Select a cointegrated pair from earlier analysis
# 2. Define rolling windows (e.g., 252 trading days = 1 year)
# 3. Calculate p-values for each window
# 4. Track hedge ratio over time
# 5. Visualize results

# Your code here

In [ ]:
# TODO: Create visualizations
# Plot 1: Rolling p-values over time (with 0.05 threshold line)
# Plot 2: Rolling hedge ratio over time
# Plot 3: Percentage of time pair is cointegrated (by year or quarter)

# Your code here

### Analysis Questions for Task 2

**TODO: Write your analysis here**

1. Is the cointegration relationship stable over time?
2. During which periods did cointegration break down?
3. How much does the hedge ratio vary? Should you update it regularly?
4. What market conditions might cause cointegration to fail?

---

## Task 3: Enhanced Pairs Trading Strategy

**Objective:** Improve the basic pairs trading strategy with additional features.

Implement at least TWO enhancements:

### Enhancement A: Dynamic Thresholds

Use different entry/exit thresholds based on market conditions.

In [ ]:
# TODO: Implement dynamic thresholds
# Ideas:
# - Wider thresholds during high volatility periods
# - Asymmetric entry/exit thresholds
# - Adaptive thresholds based on recent z-score distribution

# Your code here

### Enhancement B: Stop-Loss & Take-Profit

Add risk management rules to limit losses.

In [ ]:
# TODO: Implement stop-loss and take-profit
# - Exit position if z-score reaches extreme level (e.g., |z| > 3.5)
# - Exit position if spread moves against you beyond threshold
# - Implement maximum holding period

# Your code here

### Enhancement C: Position Sizing

Vary position size based on z-score magnitude or confidence.

In [ ]:
# TODO: Implement position sizing
# - Larger positions for more extreme z-scores
# - Consider recent strategy performance (scale down after losses)
# - Account for spread volatility

# Your code here

In [ ]:
# TODO: Compare strategy variants
# Backtest and compare:
# - Original strategy
# - Enhanced strategy (with your improvements)
# Create comparison showing:
# - Total return
# - Sharpe ratio
# - Max drawdown
# - Win rate
# - Number of trades

# Your code here

### Analysis Questions for Task 3

**TODO: Write your analysis here**

1. Which enhancement(s) improved performance the most?
2. Did enhancements reduce risk (drawdown, volatility)?
3. How did enhancements affect the number of trades?
4. What's the trade-off between complexity and robustness?

---

## Task 4 (Advanced/Optional): Market-Neutral Portfolio

**Objective:** Construct a market-neutral portfolio with target beta = 0.

Combine long and short positions to eliminate market exposure.

In [ ]:
# TODO: Implement market-neutral portfolio construction
# 1. Calculate individual asset betas vs market
# 2. Optimize portfolio with constraint: portfolio_beta = 0
# 3. Allow both long and short positions
# 4. Compare with long-only optimal portfolio

# Hint: Add nonlinear constraint for beta
# beta_constraint = {'type': 'eq', 'fun': lambda w: portfolio_beta(w) - 0}

# Your code here

In [ ]:
# TODO: Backtest market-neutral portfolio
# 1. Calculate portfolio returns
# 2. Calculate market returns
# 3. Show that portfolio is uncorrelated with market
# 4. Compare performance in up/down markets

# Your code here

**Analysis for Task 4:**

**TODO: Write your analysis**

1. What does the market-neutral portfolio look like (long/short positions)?
2. Is the beta truly zero? Check correlation with market.
3. How does performance compare in different market regimes?
4. What are the practical challenges of running market-neutral strategies?

---

## Submission Checklist

Your completed notebook should include:

- [ ] **Task 1 completed** - Constrained optimization with ≥2 variants
- [ ] **Task 2 completed** - Rolling cointegration analysis
- [ ] **Task 3 completed** - Enhanced pairs trading with ≥2 improvements
- [ ] **At least 5 visualizations** - Efficient frontiers, z-scores, performance plots
- [ ] **Performance comparison tables** - Clear metrics for all variants
- [ ] **Written analysis** (5-8 paragraphs) addressing all questions
- [ ] **Code comments** explaining implementation choices
- [ ] **Conclusions** about practical applicability

---

## Final Summary & Reflection

**TODO: Write your comprehensive summary here (5-8 paragraphs)**

Your summary should address:

1. **Portfolio Optimization Insights:**
   - How did adding constraints change optimal portfolios?
   - Is the historical optimal portfolio likely to be optimal going forward?
   - What are the practical limitations of mean-variance optimization?
   - How would you implement this in production?

2. **Cointegration Discoveries:**
   - Which pairs showed the strongest cointegration?
   - Did cointegration relationships remain stable over time?
   - What causes cointegration to break down?
   - How would you monitor cointegration in real-time?

3. **Beta & Risk Management:**
   - What role does beta play in portfolio construction?
   - How can you use beta for hedging?
   - Is market-neutrality always desirable?
   - What are alternatives to beta for measuring risk?

4. **Pairs Trading Performance:**
   - Did your enhanced strategy outperform the baseline?
   - What were the main sources of improvement (or deterioration)?
   - How sensitive is the strategy to parameter choices?
   - What market conditions favor pairs trading?

5. **Integration & Practical Application:**
   - How would you combine these techniques in a real trading system?
   - What additional data or tools would you need?
   - What are the main risks and how would you mitigate them?
   - Which technique (optimization, cointegration, or beta analysis) is most valuable?

6. **Comparison with Previous Methods:**
   - How does portfolio optimization compare to equal-weighting?
   - How does statistical arbitrage compare to momentum/trend strategies?
   - When would you use each approach?

7. **Key Learnings:**
   - What surprised you most in your experiments?
   - What would you do differently with more time/data?
   - How do these advanced techniques fit into the bigger picture of quantitative trading?

---

**Remember:** These are sophisticated techniques used by professional quantitative traders and hedge funds. Success requires:
- Rigorous backtesting with out-of-sample validation
- Careful risk management and position sizing
- Continuous monitoring and adjustment
- Understanding of market microstructure and costs

Focus on building intuition for when and why these methods work, not just on achieving high backtest returns. 🚀